In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [2]:

import pandas as pd
import numpy as np
import seaborn as sns
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
#validation_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_validation_data.csv.xz")
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")

X_train= training_data.drop(['id','data_type','target'],axis=1)
y_train=training_data['target']

#X_val = validation_data.drop(['id','data_type','target'],axis=1)
#y_val = validation_data['target']

X_test=tournament_data.drop(['id','data_type'],axis =  1 )


print("number of features {}".format(len(training_data.columns)-4))
#print("ratio of val to train {}".format(len(y_val)/len(y_train)))

number of features 310


In [3]:
#from pandas.core.groupby.generic import DataFrameGroupBy
#from sklearn.preprocessing import MinMaxScaler
#def get_group_stats(df: pd.DataFrame) -> pd.DataFrame:
#        for group in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]:
#            cols = [col for col in df.columns if group in col]
#            df[f"feature_{group}_mean"] = df[cols].mean(axis=1)
#            df[f"feature_{group}_std"] = df[cols].std(axis=1)
#            df[f"feature_{group}_skew"] = df[cols].skew(axis=1)
#        return df

#def min_max_stats(df: pd.DataFrame) -> pd.DataFrame:
#        mini=MinMaxScaler()
#        for group in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]:
#            df[f"feature_{group}_mean"] = mini.fit_transform(df[f"feature_{group}_mean"].values.reshape(-1,1))
#            df[f"feature_{group}_std"] = mini.fit_transform(df[f"feature_{group}_std"].values.reshape(-1,1))
#            df[f"feature_{group}_skew"] = mini.fit_transform(df[f"feature_{group}_skew"].values.reshape(-1,1))
#        return df 

import scipy
from sklearn.preprocessing import LabelEncoder

def eraencode(df,col):
    lb=LabelEncoder()
    df[col]=lb.fit_transform(df[col])
    return df

def interpscale(arr,minv,maxv):
    return np.interp(arr,(np.min(minv),np.max(maxv)),(minv,maxv))


def _neutralize(df, preds, proportion):
    scores = preds
    exposures = df.values
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()
def _normalize(df):
    X = (df.rank(method="first") - 0.5) / len(df)
    return scipy.stats.norm.ppf(X)
def normalize_and_neutralize(df, preds, feats, proportion=1.0):
    # Convert the scores to a normal distribution
    df[preds] = _normalize(df[preds])
    df[preds] = _neutralize(df, preds, feats, proportion)
    return df[preds]
def ar1(x):
    return np.corrcoef(x[:-1], x[1:])[0,1]

def autocorr_penalty(x):
    n = len(x)
    p = np.abs(ar1(x))
    return np.sqrt(1 + 2*np.sum([((n - i)/n)*p**i for i in range(1,n)]))

def smart_sharpe(x):
    return (np.mean(x)/(np.std(x, ddof=1) * autocorr_penalty(x)) * np.sqrt(12))

def numerai_sharpe(x):
    return ((np.mean(x) - 0.010415154) / np.std(x)) * np.sqrt(12)

def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]


# spread out target to 1, 2, 3, 4
train=eraencode(X_train,'era')
#val=eraencode(X_val,'era')
test=eraencode(X_test,'era')


y_train_int= (y_train *4).astype(np.int32)
#y_val_int = (y_val*4).astype(np.int32)



#trainm= min_max_stats(train)
#valm=min_max_stats(val)

In [4]:
sd = train.std().sort_values(ascending = False)
sd

era                      34.907755
feature_wisdom29          0.353596
feature_charisma83        0.353596
feature_intelligence4     0.353596
feature_intelligence1     0.353596
                           ...    
feature_strength17        0.284223
feature_charisma47        0.265371
feature_charisma25        0.265371
feature_strength38        0.180383
feature_strength20        0.180383
Length: 311, dtype: float64

In [5]:
highsdcol=sd[sd > 0.345].index

In [6]:
train2=train[highsdcol]
#val2=val[highsdcol]
test2=test[highsdcol]

In [7]:
!pip install catboost lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor as lgb
from sklearn.model_selection import KFold, GridSearchCV
%timeit
def ensemble_train_val_predict(init_paramsc,init_paramsl,gridc,gridl,X_train,y_train,X_test,Npochs = 1,N_Fold= 10,cvl=4,modelsb=True):
    
    catjawn=CatBoostRegressor(**init_paramsc)
    lg = lgb(**init_paramsl)
    
    if modelsb:
        models= [ ]
    
    pred_c=np.zeros(len(X_test['era'].values.reshape(-1,1)))
    pred_l=np.zeros( len(X_test['era'].values.reshape( -1,1)))
    
    k = KFold(N_Fold)
    
    for i in range(Npochs):
        for fold_, (tr_idx,val_idx) in enumerate(k.split(X_train,y_train)):
            
           # gcv = catjawn.grid_search(gridc, X_train,y_train,
                                   #   cv=(tr_idx,val_idx))
            
            glg= GridSearchCV(estimator=lg,param_grid=gridl)
            glg.fit(X_train.iloc[tr_idx],y_train.iloc[tr_idx])
       #     
       #     print("for CatBoostReg @ {} Fold the Training Corr is { }".format(fold_,correlation_score(y_train[tr_idx],gcv.predict(X_train[tr_idx]))))
      #      print("for CatBoostReg @ {} Fold the Val Corr is {}".format(fold_,correlation_score(y_train[val_idx],gcv.predict(X_train[val_idx]))))

            print(" for light gbm @ {} Fold the Training Corr is { }".format(fold_,correlation_score(y_train[tr_idx],glg.predict(X_train[tr_idx]))))
            print("for light gbm @ {} Fold the Val Corr is {}".format(fold_,correlation_score(y_train[val_idx],glg.predict(X_train[val_idx]))))

            

            
      #      pred_c+= gcv.predict(X_test)/N_Fold
            pred_l+= glg.predict(X_test)/N_Fold
            if modelsb:
              #  models.append(catjawn)
                models.append(glg)
    
 ##   pred1=np.mean ([pred_c,pred_l],axis = 0)
    pred2=interpscale(pred_l,0 ,1 )
    return models, pred2



init_paramsl = {
    'metric':'mse',
    'max_depth': 10
}

init_paramsc={ 'loss_function':'RMSE'}

gridc = {
    'learning_rate': [0.03, 0.01, 0.03, 0.009],
   'bootstrap_type':['Bayesian'],
    'bagging_temperature': [0, 1, 10]}

gridl ={'reg_alpha': [0.1, 0.5],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
    }


ensemble_train_val_predict(init_paramsc,init_paramsl,gridc,gridl,train2,y_train.astype('float'),test2)
